# NLR biology ワークショップ (2024 冬)

## バイオインフォマティクス実習

# ゲノムアセンブル
　近年、シロイヌナズナやイネなどのようなモデル生物だけでなく、さまざまな生物でゲノム配列の整備（ゲノム解読）が進められています。解読されたゲノム配列は、進化学的研究や育種学的研究など、幅広い研究の基盤情報になります。

　次世代シーケンシング（ショートリードシーケンシング）や第三世代シーケンシング（ロングリードシーケンシング）をおこなうと、元々のゲノム配列に由来する、短い塩基配列（**リード、Read**）を多数得られます。ゲノム解読は、多数の短い配列をつなぎ、元のゲノム配列を復元する試みです。その復元処理は**ゲノムアセンブル（Genome assemble）**と呼ばれています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/assemble_overview.png?raw=true" height="100px" align="middle">

　ショートリードとロングリードの違いが知りたい方は[ゲノム情報解析入門の講義](https://colab.research.google.com/github/CropEvol/lecture/blob/master/textbook_2024/Genome_assemble.ipynb)の資料等を参考にしてみてください。

　今回の実習では、午前中の実習で行ったNanopore シーケンサーによって得られたロングリードからゲノム配列をアセンブルしてみます。




### データのダウンロード
　この実習で使用するサンプルデータをダウンロードします。

Nanoporeシーケンサーによって読まれた配列データはFastq形式というファイルで保存されます。

- `bctg.fastq.gz`


In [ ]:
%%bash
wget -q -O 'get_dataset.sh' https://github.com/slt666666/NLR_biology_workshop_2024/raw/master/script/get_dataset.sh
bash get_dataset.sh
ls | grep -e 'fastq.gz'

### ソフトウェアのダウンロード

　今回使用するソフトウェアとその実行プログラムをダウンロードをします。

ナノポアから得られるロングリードをアセンブルするソフトウェアには様々なものがあり、今でも新たなソフトが開発されています。

本実習では[`Flye`](https://github.com/mikolmogorov/Flye)というソフトを使用します。



In [ ]:
%%bash
wget -q -O 'get_software.sh' https://github.com/slt666666/NLR_biology_workshop_2024/raw/master/script/get_software.sh
bash get_software.sh
ls tools/

### 今回の実習内容

1. ロングリードを使ったゲノムアセンブル
 1. ロングリードを確認する
 1. ゲノムアセンブル
 1. ロングリードの特徴







---
## ロングリードを使ったゲノムアセンブル

　ショートリードシーケンシングは、50-300 bpの短い塩基配列を多量に得るシーケンシングです。近年、非常に長い塩基配列を得るシーケンシング技術（**ロングリードシーケンシング、Long read sequencing**）が発達してきました。Oxford Nanopore Technologies社のナノポアシーケンシングや、Pacific Biosciences社のPacBioシーケンシングがそれにあたります。

　そのため、現在では、ロングリードを使ったゲノムアセンブルが最もよくおこなわれています。ここでは、ナノポアシーケンシングによるロングリードからどのようなゲノム配列を復元できるかを見ていきましょう。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/long_read_assemble.png?raw=true" height="100px" align="middle">

### 2-1. ロングリードを確認する

　ロングリードのシーケンスデータを見てみましょう。

#### リードのデータファイル形式（FASTQ）

　リードのデータは**FASTQ**と呼ばれる形式でファイルに保存されています。FASTQ形式では、4行で1つのリードデータの情報を表しています。

```
@リードの名前
TGAAGCAGT..............(リードの塩基配列)
+
F1GCFCGGE..............(各塩基のクオリティ)
```

- 第1行（@からはじまる行）: リードの名前
- 第2行: リードの塩基配列
- 第3行（+からはじまる行）: 第2行と第4行の区切りのための行
- 第4行: 各塩基配列のクオリティ（ASCIIコードで書かれている）。1文字が何らかのクオリティスコア（数値）を示している。
 - 参考: https://bi.biopapyrus.jp/rnaseq/qc/fastq-quality-score.html



In [ ]:
# fastqファイルの一部を表示
!head bctg.part_001.fastq

#### seqkitを使ったリードの統計情報確認

 　[seqkit](https://bioinf.shenwei.me/seqkit/)というソフトウェアを使って、午前中に読んだ配列の統計情報（リードの数やリードの長さなどの情報）を確認してみましょう。

```sh
# リードの統計情報の確認
seqkit stat 塩基配列データ
```

In [ ]:
%%bash
seqkit stat bctg.fastq.gz

### 2-2. ゲノムアセンブル
　ロングリードを使ったアセンブルを行うことが出来るソフトウェアは様々なものが存在しており、また日々開発が進んでいます。

* [Flye](https://github.com/fenderglass/Flye)
* [Necat](https://github.com/xiaochuanle/NECAT)
* [NextDenovo](https://github.com/Nextomics/NextDenovo)
* 等々

今回はロングリード用のソフトウェア[Flye](https://github.com/fenderglass/Flye)を使って、ゲノムアセンブルを試みてみましょう。




In [ ]:
%%bash
## アセンブル
bash Flye_assemble.sh bctg.fastq.gz


　アセンブリは、次のファイルに保存されます。

/content/flye_output/assembly.fasta

　seqkitで、アセンブリの統計情報を確認してみましょう。

In [ ]:
%%bash
seqkit stat -a /content/flye_output/assembly.fasta

---
## まとめ・補足

　今回、ロングリードを使ってゲノムアセンブルをおこない、ゲノム配列の復元を試みました。

　ロングリードのシーケンスが普及したことで、非常に長いリードをアセンブルに利用出来るようになり、ゲノム解読の難易度は劇的に低下しました。

特に本実習で扱った菌類などのゲノムサイズが比較的小さい生物であれば、その日の内にゲノムを決めることが出来ます。

<br>

### 補足:ナノポアのエラーについて

Nanoporeのロングリードは一部エラー率が高いリードが含まれる可能性があり、部分的には塩基情報が不正確なコンティグが生成されることがあります。

　その様な場合、精度の高いショートリードと長い配列のロングリードそれぞれの利点を活かすことで、より良いゲノム配列を構築することができます。ロングリードで得られた長いコンティグを、ショートリードの正確性で補い、高精度なゲノム配列を得る形です。このようなロングリードとショートリードを組み合わせたゲノム解読が、現在ではよく使われています。

<img src="https://github.com/CropEvol/lecture/blob/master/textbook_2023/images/Assemble_figs/major_assemble_proc.png?raw=true" height="220px" align="middle">